In [32]:
:ext GeneralizedNewtypeDeriving
:ext BlockArguments

### Introduction to contravariant logging

We decided to use contravariant logging approach. I use here `co-log` lib naming convention because that is what we have in the code base now (`cardano-node` uses `contra-tracer`).

The core type is just a logging function `msg -> m ()` (for example `Text -> IO ()`). We wrap in a `newtype` and call it `LogAction`:

In [33]:
newtype LogAction m msg = LogAction (msg -> m ())

For example `putStrLn` can be easily turned into a log action and it is `String` logging action in `IO`:

In [34]:
consoleLogAction :: LogAction IO String
consoleLogAction = LogAction putStrLn

We can also easily implement a file logger (but please don't use it in production ;-):

In [35]:
fileLogAction :: FilePath -> LogAction IO String
fileLogAction filePath = LogAction \msg -> appendFile filePath (msg <> "\n")

#### Combining actions "veritically"

> Note: I used "vertically" in the title because we can imagine that we have a logging pipeline - `log entry -> log transformation / adjustment -> log consumers`. From the application point of view we have a single entry point to this pipeline.

If we have loggers which work with the same message type (`String` in our case) we have pretty natural way of combining we are nearly able to use `deriving newtype` for that and use `->` and `m ()` underlying instances but let's write this composition by hand...
I mean please implement this composition: 

In [36]:
instance (Applicative m) => Semigroup (LogAction m msg) where
  LogAction log1 <> LogAction log2 = LogAction \msg -> log1 msg *> log2 msg
  
instance (Applicative m) => Monoid (LogAction m msg) where
  mempty = LogAction \msg -> pure ()